In [ ]:
#1 - E
#2 - I would choose scenerio 1 because the validation error is only slightly higher than the
#training error, which means that it was trained well on the dataset
#3 - outliers are one of the main differences. gradient boosting is stronger with outliers
#4 - gradient boosting assumes each tree could miss and uses ensemble. random forest assumes that the
#trees make good predictions
#5 - A

In [21]:
import boto3 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler

#defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data445-ashlyn-bucket'
bucket = s3.Bucket(bucket_name)

#defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [22]:
#input and target variables
X = heart[['age', 'totChol','sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

#splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)

In [23]:
#transforming inputs to 0-1 scale
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [24]:
#random forest
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

#predicting test data
pred1 = rf_md.predict_proba(X_test)[:, 1]
pred1_label = np.where(pred1 < 0.1, 0, 1)

#computing the recall
recall_score(Y_test, pred1_label)

0.9464285714285714

In [25]:
#accuracy score of rf_md
accuracy_score(Y_test, pred1_label)

0.3442622950819672

In [26]:
#adaboost model
ab_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

#predicting test data
pred2 = ab_md.predict_proba(X_test)[:, 1]
pred2_label = np.where(pred2 < 0.1, 0, 1)

#computing the recall
recall_score(Y_test, pred2_label)

0.9732142857142857

In [27]:
#accuracy score of ab_md
accuracy_score(Y_test, pred2_label)

0.1871584699453552

In [28]:
#gradient boosting model
gb_md = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

#making predictions
pred3 = gb_md.predict_proba(X_test)[:, 1]
pred3_label = np.where(pred3 < .1, 0, 1)

#recall score
recall_score(Y_test, pred3_label)

0.9107142857142857

In [29]:
#accuracy score of gb_md
accuracy_score(Y_test, pred3_label)

0.41530054644808745

In [30]:
final_md1 = []
final_md2 = []
final_md3 = []
accuracy_md1 = []
accuracy_md2 = []
accuracy_md3 = []

#repeating 100 times
for i in range(0, 100):
    
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)
    
    rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    ab_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.1).fit(X_train, Y_train)
    gb_md = GradientBoostingClassifier(max_depth = 3, n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)
    
    pred1 = rf_md.predict_proba(X_test)[:, 1]
    pred1 = np.where(pred1 < .1, 0, 1)

    pred2 = ab_md.predict_proba(X_test)[:, 1]
    pred2_label = np.where(pred2 < .1, 0, 1)

    pred3 = gb_md.predict_proba(X_test)[:, 1]
    pred3_label = np.where(pred3 < .1, 0, 1)
   
    final_md1.append(recall_score(Y_test, pred1_label))
    final_md2.append(recall_score(Y_test, pred2_label))
    final_md3.append(recall_score(Y_test, pred3_label))
    accuracy_md1.append(accuracy_score(Y_test, pred1_label))
    accuracy_md2.append(accuracy_score(Y_test, pred2_label))
    accuracy_md3.append(accuracy_score(Y_test, pred3_label))
    
print('Model 1 avg recall score is:', np.mean(final_md1))
print('Model 2 avg recall score is:', np.mean(final_md2))
print('Model 3 avg recall score is:', np.mean(final_md3))
print('Model 1 avg accuracy score is:', np.mean(accuracy_md1))
print('Model 2 avg accuracy score is:', np.mean(accuracy_md2))
print('Model 3 avg accuracy score is:', np.mean(accuracy_md3))

Model 1 avg recall score is: 0.7917857142857143
Model 2 avg recall score is: 1.0
Model 3 avg recall score is: 0.811160714285714
Model 1 avg accuracy score is: 0.2969398907103825
Model 2 avg accuracy score is: 0.15323770491803274
Model 3 avg accuracy score is: 0.503688524590164


In [ ]:
#based on the results above, I would choose model 3 because it has the highest avg recall and avg accuracy scores

In [ ]:
#none of the models had both scores greater than or equal to 80%. To fix this I would 